## Comments/Explaination/Discussion on the performance of the Frameworks

## Training Time

 TODO: rephrase/clarify a few paragraphs

![trainingtimeresults](FAQ-images/trainingtime.png)

<div style="text-align: justify"> The OriginalC code is written purely in C and is the only framework that makes use of a single language. All other frameworks, exploit low level languages for computationally expensive parts of the algorithm. For instance, Gensim’s implementation has a highly optimized code realized using Cython and calls to BLAS routines yet maintains the Pythonic interface.  
DL4J uses Java for reading/tokenizing words while all the mathematics (matrix multiplication etc.) is handled in C++. Tensorflow is implemented using operations in Python and input processing in C++. Given this information about the implementations, the surprising part isn’t the ranking between these frameworks but rather the huge gap in the training times.  
While Gensim’s performance is competitive to the Original-C implementation, the same cannot be said about DL4J and Tensorflow. Gensim’s and DL4J’s implementation are specifically designed to improve performance on the Word2Vec algorithm. On the other hand, Tensorflow was developed with an intent to support numerical computations using data flow graphs which can be deployed almost anywhere(desktops, servers, mobile devices). It isn’t specially tuned to handle the Word2Vec algorithm.  
One would also expect tensorflow-gpu to be faster than tensorflow running on cpu but the results show otherwise. There seem to be many discussions on the web around this topic. It is generally seen that small/shallow networks train faster on CPU(s) compared to a single GPU. Embeddings, by their nature, tend to emphasize fine-grained, random memory lookups. That plays much less to the strengths of the GPU which results in the underutilisation of GPU (~ 20-50%). Also, the constant data transfer between CPU & GPU slows things further. [Here](https://rare-technologies.com/gensim-word2vec-on-cpu-faster-than-word2veckeras-on-gpu-incubator-student-blog/) is a blog by Šimon Pavlík (RaRe Incubator graduate), which discusses this issue in more detail.  

Another interesting observation from the above box plot is the spread of each framework’s training times. It is difficult to get a true picture of the dispersion because of wide differences in each framework’s mean training time. Let us have a look at their coefficient of variation. 
</div>

|  Framework | Coefficient of Variation  (Relative training times) |
|:----------:|:--------------------------------------------------:|
| Original-C |                         0.0                        |
|   Gensim   |                        0.38                        |
|    DL4J    |                         0.6                        |
| Tensorflow |                        0.61                        |

<div style="text-align: justify">
These values suggest that each framework’s relative training time fluctuates non trivially with an instance’s computational power (number of CPU cores & clock speed). In other words, the proportional benefit derived with higher CPU cores is different for each framework. Say for example, doubling cores doubled the training speed and thus halved the training time. If all frameworks were to profit from this equally, the training time for every framework would be halved resulting in the same relative (to Original-C) training time. In such a case the coefficient of variations would be 0 for every framework.  
More evidence in the scatter plot below:
</div>



## Peak Memory Utilization

![peakmemoryresults](FAQ-images/peakmemory.png)

<div style="text-align: justify">The Original C and Gensim process a constant number of words before explicitly getting rid of them from memory and moving on to the next sequence of words. This is primarily the reason they have a much smaller memory footprint compared to DL4J and Tensorflow.  In DL4J’s implementation, a line is assumed to be a sentence. Co-occurrence statistics are gathered line by line before running the skipgram cycle. In our text corpus, each line can contain up to 10,000 words and therefore the co-occurrence statistics for these can consume significant memory.   
Another observation is the wide spread in the values for DL4J’s memory consumption. This can be explained by uncertainty in the order of execution of threads (where each thread takes up significant RAM to store sentence level statistics) together with the unpredictability of the Garbage Collector (Java’s automatic memory management).   
Tensorflow too has a high memory footprint because in spite of processing in small constant sized batches, it does not explicitly “free” the memory it uses.  
</div>